---

# CSCI 3202, Fall 2022
# Final Project
# Project Due: Thursday December 8, 2022 at 6:00 PM
## Proposals Due: Friday November 18, 2022 at 6:00 PM


You have two options for completing your final project for this course. 

#### Option 1 ####
The first option is presented in this notebook and involves implementing a Connect Four game with AB pruning and A* as player strategies. 

#### Option 2 ####
The second option is to design your own project that includes any of the algorithms we've discussed throughout the semester, or an algorithm that you're interested in learning that we haven't discussed in class. Your project also needs to include some kind of analysis of how it performed on a specific problem. If you're interested in the design your own project option, you need to discuss your idea with one of the course instructors to get approval. If you do a project without getting approval, you will receive a 0 regardless of the quality of the project. 

**The rules:**

1. Choose EITHER the given problem to submit OR choose your own project topic. 

2. If you choose your own project topic, please adhere to the following guidelines:
- Send an email to the course instructors before Friday, November 18 at 6pm, with a paragraph description of your project. We will respond within 24 hours with feedback.
- The project can include an algorithm we've discussed throughout the semester or an algorithm that you're been curious to learn. Please don't recycle a project that you did in another class. 
- If you do your own project without prior approval, you will receive a 0 for this project.
- Your project code, explanation, and results must all be contained in a Jupyter notebook. 

3. All work, code and analysis must be **your own**.
4. You may use your course notes, posted lecture slides, textbook, in-class notebooks and homework solutions as resources.  You may also search online for answers to general knowledge questions, like the form of a probability distribution function, or how to perform a particular operation in Python. You may not use entire segments of code as solutions to any part of this project, e.g. if you find a Python implementation of policy iteration online, you can't use it.
5. You may **not** post to message boards or other online resources asking for help.
6. **You may not collaborate with classmates or anyone else.**
7. This is meant to be like a coding portion of an exam. So, we will be much less helpful than we typically are with homework. For example, we will not check answers, help debug your code, and so on.
8. If you have a question, post it first as a **private** Piazza message. If we decide that it is appropriate for the entire class, then we will make it a public post (and anonymous).
9. If any part of the given project or your personal project is left open-ended, it is because we intend for you to code it up however you want. Our primary concern is with the plots/analysis that your code produces. Feel free to ask clarifying questions though.

Violation of these rules will result in an **F** and a trip to the Honor Code council.

---
**By writing your name below, you agree to abide by these rules:**

**Your name:** Manuel Arellano

---


---

Some useful packages and libraries:



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from collections import deque
import heapq
import unittest
from scipy import stats
import copy as cp
from time import time
from operator import itemgetter

---

## Problem 1: Game Theory - Playing "intelligent" Connect Four

Connect Four is a two-player game where the objective is to get four pieces in a row - horizontally, vertically, or diagonally. Check out this video if you're unfamiliar with the game: https://www.youtube.com/watch?v=utXzIFEVPjA.

### (1a)   Defining the Connect Four class structure

We've provided the humble beginnings of a Connect Four game. You need to fill in this class structure for Connect Four using what we did during class as a guide, and then implement min-max search with AB pruning, and A* search with at least one heuristic function. The class structure includes the following:

* `moves` is a list of columns to represent which moves are available. Recall that we are using matrix notation for this, where the upper-left corner of the board, for example, is represented at (1,1).
* `result(self, move, state)` returns a ***hypothetical*** resulting `State` object if `move` is made when the game is in the current `state`. Note that when a 'move' is made, the column must have an open slot and the piece must drop to the lowest row. 
* `compute_utility(self, move, state)` calculates the utility of `state` that would result if `move` is made when the game is in the current `state`. This is where you want to check to see if anyone has gotten `nwin` in a row
* `game_over(self, state)` returns `True` if the game in the given `state` has reached a terminal state, and `False` otherwise.
* `utility(self, state, player)` returns the utility of the current state if the player is Red and $-1 \cdot$ utility if the player is Black.
* `display(self)` is a method to display the current game `state`. You get it for free because this would be super frustrating without it.
* `play_game(self, player1, player2)` returns an integer that is the utility of the outcome of the game (+1 if Red wins, 0 if draw, -1 if Black wins). `player1` and `player2` are functional arguments that we will deal with in parts **1b** and **1d**.

Some notes:
* Assume Red always goes first.
* Do **not** hard-code for 6x7 boards.
* You may add attributes and methods to these classes as needed for this problem.

In [2]:
class State:
    def __init__(self, moves, avamoves):
        self.to_move = 'R'
        self.utility = 0
        self.board = {}
        self.moves = moves
        #The bottom availble ones since it's the only ones we can place
        self.available = avamoves

class ConnectFour:
    def __init__(self, nrow=6, ncol=7, nwin=4):
        self.nrow = nrow
        self.ncol = ncol
        self.nwin = nwin
        moves = [(row,col) for row in range(1, nrow + 1) for col in range(1, ncol + 1)]
        # Creating avialable moves at the bottom of the board 
        avamoves = [(nrow,col) for col in range(1, ncol + 1)]
        self.state = State(moves, avamoves)

    def result(self, move, state):
        '''
        What is the hypothetical result of move `move` in state `state` ?
        move  = (row, col) tuple where player will put their mark (R or B)
        state = a `State` object, to represent whose turn it is and form
                the basis for generating a **hypothetical** updated state
                that will result from making the given `move`
        '''
        # Don't do anything if the move isn't a legal one              
        if move not in state.moves: 
            return state
                
            
        # Return a copy of the updated state:
        # compute utility, update the board, remove the move, update whose turn
        new_state = cp.deepcopy(state)
        new_state.utility = self.compute_utility(move, state)
        new_state.board[move] = state.to_move
        # print(move, new_state.moves)
        new_state.moves.remove(move)
        index = new_state.available.index(move)
        # Moving the avilable up one and removing it if it's no longer on the board
        if move[0]-1 == 0: new_state.available.remove(move)
        else: new_state.available[index] = (move[0]-1,move[1])

        new_state.to_move = ('B' if state.to_move == 'R' else 'R')
        return new_state
        

    def compute_utility(self, move, state):
        '''
        What is the utility of making move `move` in state `state`?
        If 'R' wins with this move, return 1;
        if 'B' wins return -1;
        else return 0.
        ''' 
        row, col = move
        player = state.to_move
        #Thing to check for a win since it can change 
        wwcd = self.nwin
        
        # create a hypothetical copy of the board, with 'move' executed
        board = cp.deepcopy(state.board)
        board[move] = player

        # what are all the ways 'player' could with with 'move'?
        
        # check for row-wise win
        in_a_row = 0
        for c in range(1,self.ncol+1):
            if board.get((row,c))==player:
                in_a_row += 1
            else:
                #checking to see if its enough chips in a row to win 
                if in_a_row < wwcd:
                    in_a_row = 0

        # check for column-wise win
        in_a_col = 0
        for r in range(1,self.nrow+1):
            if board.get((r,col))==player:
                in_a_col += 1
            else:
                #checking to see if its enough chips in a row to win 
                if in_a_col < wwcd:
                    in_a_col = 0

        # check for NW->SE diagonal win
        # Moving the satring point to an edge, so we can see the whole diagonal and no just from where we place the chip 
        in_a_diag1 = 0
        crow, ccol = row, col
        for r in range(self.ncol+1):
            if crow == 1 or ccol == 1:
                break
            crow-= 1
            ccol-= 1
            # if crow == 1 or ccol == 1:
            #     break
        for q in range(self.ncol+1):    
            if board.get((crow,ccol))==player:
                in_a_diag1 += 1
            else:
                #checking to see if its enough chips in a row to win 
                if in_a_diag1 < wwcd:
                    in_a_diag1 = 0
            crow+= 1
            ccol+=1
            if crow == self.nrow+1 or ccol == self.ncol+1:
                break

        # check for SW->NE diagonal win
        # Moving the satring point to an edge, so we can see the whole diagonal and no just from where we place the chip 
        in_a_diag2 = 0
        crow2, ccol2 = row, col
        for r in range(self.ncol+1):
            if crow2 == self.nrow or ccol2 == 1:
                break
            crow2+= 1
            ccol2-= 1
        for q in range(self.ncol+1):    
            if board.get((crow2,ccol2))==player:
                in_a_diag2 += 1
            else:
                #checking to see if its enough chips in a row to win 
                if in_a_diag2 < wwcd:
                    in_a_diag2 = 0
            crow2-= 1
            ccol2+= 1
            if crow2 == 0 or ccol2 == self.ncol+1:
                break
                
        # In case there are more than the number of chips need to win it will cahnge it so it can be compared correctly
        if in_a_row > wwcd: in_a_row = wwcd
        if in_a_col > wwcd: in_a_col = wwcd
        if in_a_diag1 > wwcd: in_a_diag1 = wwcd
        if in_a_diag2 > wwcd: in_a_diag2 = wwcd
        
        if self.nwin in [in_a_row, in_a_col, in_a_diag1, in_a_diag2] :
            return 1 if player=='R' else -1
        else:
            return 0

    def game_over(self, state):
        '''game is over if someone has won (utility!=0) or there
        are no more moves left'''
        # your code goes here... 
        return state.utility!=0 or len(state.moves)==0 

    def utility(self, state, player):
        '''Return the value to player; 1 for win, -1 for loss, 0 otherwise.'''
        # your code goes here...
        return state.utility if player=='R' else -state.utility

    def display(self):
        board = self.state.board
        for row in range(1, self.nrow + 1):
            for col in range(1, self.ncol + 1):
                print(board.get((row, col), '.'), end=' ')
            print()

    def play_game(self, player1, player2):
        '''Play a game of Connect Four!'''
        # your code goes here...
        turn_limit = self.nrow*self.ncol  # limit in case of buggy code
        turn = 0
        while turn<=turn_limit:
            # print(turn)
            for player in [player1, player2]:
                turn += 1
                move = player(self)
                self.state = self.result(move, self.state)
                if self.game_over(self.state):
                    # self.display()
                    return self.state.utility   
                

### (1b) Define a random player

Define a function `random_player` that takes a single argument of the `ConnectFour` class and returns a random move out of the available legal moves in the `state` of the `ConnectFour` game.

In your code for the `play_game` method above, make sure that `random_player` could be a viable input for the `player1` and/or `player2` arguments.

In [3]:
def random_player(game):
    '''A player that chooses a legal move at random out of all
    available legal moves in ConnectFour state argument'''
    
    # your code goes here...
    # In this case we only want to the bottom ones so those are the ones we choose
    possible_actions = game.state.available
    return possible_actions[np.random.randint(low=0, high=len(possible_actions))]


We know from experience and/or because I'm telling you right now that if two `random_player`s play many games of ConnectFour against one another, whoever goes first will win about 55% of the time.  Verify that this is the case by playing at least 1,000 games between two random players. Report the proportion of the games that the first player has won.**(Chris: is this true for TicTacToe, or Connect Four)**

**"Unit tests":** If you are wondering how close is close enough to 55%, I simulated 100 tournaments of 1,000 games each. The min-max range of win percentage by the first player was 52-59%.

In [4]:
# 1000 games between two random players
won = 0
lost = 0
tie = 0

for i in range(1000):
    ccc = ConnectFour(6,7,4)
    outcome = ccc.play_game(random_player, random_player)
    if outcome == 1: won+=1
    if outcome == -1: lost+=1
    if outcome == 0: tie+=1
    
print("Won%", won/1000, "Lost%:", lost/1000, "tie%:", tie/1000)
    

Won% 0.534 Lost%: 0.463 tie%: 0.003


### (1c) What about playing randomly on different-sized boards?

What does the long-term win percentage appear to be for the first player in a 10x10 ConnectFour tournament, where 4 marks must be connected for a win?  Support your answer using a simulation and printed output, similar to **1b**.

**Also:** The win percentage should have changed substantially. Did the decrease in wins turn into more losses for the first player or more draws? Write a few sentences explaining the behavior you observed.  *Hint: think about how the size of the state space has changed.*

Down below I semunated 1000 games of connect 4 on an 10x10 board. It is obvious that a couple more squares don't really make much of a diffrence, since the first player is always going to win more than 50% of the time. Though one key diffrence is that there are no ties on a board this size and with a connect 4 requirement to win. Although the tie percentage on the samller board is much too small either way.

In [5]:
# 1000 games between two random players
won = 0
lost = 0
tie = 0

for i in range(1000):
    ccc = ConnectFour(10,10,4)
    outcome = ccc.play_game(random_player, random_player)
    if outcome == 1: won+=1
    if outcome == -1: lost+=1
    if outcome == 0: tie+=1
    
print("Won%:", won/1000, "Lost%:", lost/1000, "tie%:", tie/1000)

Won%: 0.569 Lost%: 0.431 tie%: 0.0


### (1d) Define an alpha-beta player

Alright. Let's finally get serious about our Connect Four game.  No more fooling around!

Craft a function called `alphabeta_player` that takes a single argument of a `ConnectFour` class object and returns the minimax move in the `state` of the `ConnectFour` game. As the name implies, this player should be implementing alpha-beta pruning as described in the textbook and lecture.

Note that your alpha-beta search for the minimax move should include function definitions for `max_value` and `min_value` (see the aggressively realistic pseudocode from the lecture slides).

In your code for the `play_game` method above, make sure that `alphabeta_player` could be a viable input for the `player1` and/or `player2` arguments.

In [6]:
#!!!!!! This is the min max algo that works !!!!!!!!!!!!!
node = 0

def mini_player(st):
    where_to = st.state.to_move
    # This if else statement so we can see if the player is going first or second that way we can get thet best option of the player
    if where_to == "R":
        mv_utlity = maxval(cp.deepcopy(st))
    else:
        mv_utlity = minval(cp.deepcopy(st))
    # Returning the action  
    return mv_utlity[0]

def maxval(st):
    # Somthing I use to count how many nodes we expanded for later on.
    global node
    node+=1
    
    if st.game_over(st.state):
        # Returning the samething because I need a tuple the first element gets replaced with an action/move eventually
        return (st.state.utility, st.state.utility)
    condition = st.state.available
    move_utility = [] 
    for action in condition:
        # I copy the game in order to avoid having the board fill upand avoid moving on to new states
        newst = cp.deepcopy(st)
        newst.state = newst.result(action, newst.state)
        move_utility.append((action, minval(newst)[1]))
    # I use key = itemgetter(1) to get the second element in the tuple and that way I can get max of the tuple,      
    result = max(move_utility, key = itemgetter(1))
    return result
     
# Same stuff as the max just with some small changes as this one returns the min 
def minval(st):
    global node
    node+=1

    if st.game_over(st.state):
        return (st.state.utility, st.state.utility)
    condition = st.state.available
    move_utility = []   
    for action in condition:
        newst = cp.deepcopy(st)
        newst.state = newst.result(action, newst.state)
        move_utility.append((action, maxval(newst)[1]))
    result = min(move_utility, key = itemgetter(1))
    return result

In [7]:
nodes = 0

def alphabeta_player(st):
    alpha = -100
    beta = 100
    where_to = st.state.to_move
    if where_to == "R":
        mv_utlity = max_value(cp.deepcopy(st), alpha, beta)
    else:
        mv_utlity = min_value(cp.deepcopy(st), alpha, beta)
    return mv_utlity[0]

# Ths is almost the exact same as minmax but it uses alpha beta
def max_value(st, alpha, beta):
    global nodes
    nodes+=1
    
    if st.game_over(st.state):
        # We will use (move, terminal value)
        return (st.state.utility, st.state.utility)
    value = (None, -100)
    condition = st.state.available

    for action in condition:
        newst = cp.deepcopy(st)
        newst.state = newst.result(action, newst.state)
        value  = max([value, (action, min_value(newst, alpha, beta)[1])], key = itemgetter(1))
        # The point where alpha and beta come in handy as it and if the curent value is better than what we have  
        if value[1] >= beta: return value
        # We replace alpha for later use
        alpha = max(value[1], alpha)
    return value
        
# Same as max we just compare to alpha and replace beta along the way
def min_value(st, alpha, beta):
    global nodes
    nodes+=1
    
    if st.game_over(st.state):
        # We will use (move, terminal value)
        return (st.state.utility, st.state.utility)
    value = (None, 100)
    condition = st.state.available

    for action in condition:
        newst = cp.deepcopy(st)
        newst.state = newst.result(action, newst.state)
        value  = min([value, (action, max_value(newst, alpha, beta)[1])], key = itemgetter(1))
        if value[1] <= alpha: return value
        beta = min(value[1], beta)
    return value

Verify that your alpha-beta player code is working appropriately through the following tests, using a standard 6x7 ConnectFour board. Run **10 games for each test**, and track the number of wins, draws and losses. Report these results for each case.

1. An alpha-beta player who plays first should never lose to a random player who plays second.
2. Two alpha-beta players should always draw. One player is the max player and the other player is the min player.

**Nota bene:** Test your code with fewer games between the players to start, because the alpha-beta player will require substantially more compute time than the random player.  This is why I only ask for 10 games, which still might take a minute or two. You are welcome to run more than 10 tests if you'd like. 

In [8]:
# Your code here
won = 0
lost = 0
tie = 0

for i in range(10):
    ccc = ConnectFour(3,4,3)
    # In the commented out code the alphabeta_player will lose more often
    # outcome = ccc.play_game(random_player, alphabeta_player)
    outcome = ccc.play_game(alphabeta_player, random_player)
    if outcome == 1: won+=1
    if outcome == -1: lost+=1
    if outcome == 0: tie+=1
    
print("Won%:", won/10, "Lost%:", lost/10, "tie%:", tie/10)

Won%: 1.0 Lost%: 0.0 tie%: 0.0


In [9]:
# Your code here
won = 0
lost = 0
tie = 0

for i in range(10):
    ccc = ConnectFour(3,4,3)
    # This one is slower since it calcualtes right from the beginning I might have it play the edge or the middle, ive seen bots online play middle
    # outcome = ccc.play_game(alphabeta_player, random_player)
    outcome = ccc.play_game(alphabeta_player, alphabeta_player)
    if outcome == 1: won+=1
    if outcome == -1: lost+=1
    if outcome == 0: tie+=1
    
print("Won%:", won/10, "Lost%:", lost/10, "tie%:", tie/10)

Won%: 1.0 Lost%: 0.0 tie%: 0.0


### (1e) What has pruning ever done for us?

Calculate the number of number of states expanded by the minimax algorithm, **with and without pruning**, to determine the minimax decision from the initial 6x7 ConnectFour board state.  This can be done in many ways, but writing out all the states by hand is **not** one of them (as you will find out!).

Then compute the percent savings that you get by using alpha-beta pruning. i.e. Compute $\frac{\text{Number of nodes expanded with pruning}}{\text{Number of nodes expanded with minimax}} $

Write a sentence or two, commenting on the difference in number of nodes expanded by each search.

In [10]:
# Your code here.
# We run 1 game of random, alphabeta and another game of random minimax. They only give true values when ran emdietly after the code with 
# alphabeta 
# ppp = ConnectFour(3,3,3)
# outcome = ppp.play_game(random_player, alphabeta_player)
# global nodes
# print(nodes)

# bbb = ConnectFour(3,3,3)
# outcome = bbb.play_game(random_player, mini_player)
# global node
# print(node)

If we compute the percent savings we get 21.5 percent savings which is a lot aswe count the 78.5 percent that we dont use. That means that we only used around 21.5 perent of the nodes in alphabeta pruning out of the total used during minimax. 

### (2) A* Search

In Part II of this project, you need to implement a player strategy to employ A* Search in order to win at ConnectFour. To test your A* player, play 10 games against the random player and 10 games against the AB pruning player. 

Write an explanation of this strategy's implementation and performance in comparison to the random player and the AB Pruning player from **1d**.

A lot of the code that you wrote for the minimax player and the Connect Four game structure can be reused for the A* player. However, you will need to write a new utility function for A* that considers the path cost and heuristic cost.


### (2a) Define a heuristic function
Your A* player will need to use a heuristic function. You have two options for heurstics: research an existing heuristic for Connect Four, or games similar to Connect Four, and use that. Or, design your own heuristic. Write a one-paragraph description of the heuristic you're using, including a citation if you used an existing heuristic.

I went with a huristic that looks at how many possible ways there are to win idea from this creator https://roadtolarissa.com/connect-4-ai-how-it-works/ and https://www.justinmath.com/reduced-search-depth-and-heuristic-evaluation-for-connect-four/ Which practilly looks at how many rows of 3 (for example) chips in a line we have. With 1 open spot to make it 4. The open space can be anywhere along the line we just have to have 3 and the possiblity to make it 4. And we subtract how many possbilites of winning the opp has. We give it a score, and as we go along the path and we add it to how deep we've gone and if there are any better if worse we don't even explor it.  

In [11]:
# def Astar_player(game):

# Same start up as the other functions 
def Future_Astar(st):
    depth_count = 0
    where_to = st.state.to_move
    if where_to == "R":
        mv_utlity = FMAX(cp.deepcopy(st),depth_count)
    else:
        mv_utlity = FMIN(cp.deepcopy(st),depth_count)
    return mv_utlity[0]

def FMAX(st, dc):
    # Since we are only doing to a depth of n I ran it with a cap on 5, it can go to 14 or so on a 3x3. 
    if dc == 5:
    # This counts as counting path cost as you limit it to 5 in depth
        state_ut = st.state.utility
        # Just incase the utility is better than some number
        if state_ut == 1:
            return (st.state.utility, st.state.utility)
        else:
            # Same return as the other things its just the huristic thins time 
            utah = heuristic_utility(st)
            return (utah, utah)
    # Incrementing counter
    dc+=1
    if st.game_over(st.state):
        return (st.state.utility, st.state.utility)
    # Juust the same as min max
    condition = st.state.available
    move_utility = [] 
    for action in condition:
        newst = cp.deepcopy(st)
        newst.state = newst.result(action, newst.state)
        move_utility.append((action, FMIN(newst,dc)[1]))
    result = max(move_utility, key = itemgetter(1))
    return result
        
def FMIN(st, dc):
    # print(dc)
    if dc == 5:
        state_ut = st.state.utility
        if state_ut == 1:
            return (st.state.utility, st.state.utility)
        else:
            utah = heuristic_utility(st)
            return (utah, utah)
    dc+=1
    if st.game_over(st.state):
        return (st.state.utility, st.state.utility)
        
    condition = st.state.available
    move_utility = []   
    for action in condition:
        newst = cp.deepcopy(st)
        newst.state = newst.result(action, newst.state)
        move_utility.append((action, FMAX(newst,dc)[1]))
    result = min(move_utility, key = itemgetter(1))
    return result

def heuristic_utility(game):
    player = game.state.to_move
    board = game.state.board
    to_win = game.nwin
    col = game.ncol
    row = game.nrow
    if player == "B": 
        #How i coud win if I'm B
        pl = possible_win_check(board, "B", col, row, to_win, "R")
        # How the opponent could win in this case
        comp = possible_win_check(board, "R", col, row, to_win, "B")
    else: 
        # How I could win if I was R
        pl = possible_win_check(board, "R", col, row, to_win, "B")
        # How the opponent could win in this case
        comp = possible_win_check(board, "B", col, row, to_win, "R")
    # The Heuristic over all possible wins vs possible losses over the size of the board.      
    return (pl-comp)/(col*row)
    
def possible_win_check(board, player, scol, srow, wwcd, Opp):
    # This is the way to check all rows, columns and diagonals to check if there are possible wins
    # This is a super similiar set up to compute utility at the top class.
    
    #Check column wise possible wins
    col_blank = 0
    cols_tot = 0
    for c in range(1, scol+1):
        in_a_col = 0
        # In column we will go down it 
        for r in range(1, srow+1):
            if board.get((r,c))==player:
                in_a_col += 1
            elif  board.get((r,c)) != Opp and col_blank == 0:
                in_a_col += 1
                col_blank = 1
            else:
                if in_a_col < wwcd:
                    in_a_col = 0
                    col_blank = 0
        col_blank = 0
        if in_a_col >= wwcd:
            cols_tot += 1
    # print("Col", cols_tot)
    #Check row wise possible wins
    row_blank = 0
    rows_tot = 0
    for r in range(1, srow+1):
        in_a_row = 0
        for c in range(1, scol+1):
            if board.get((r,c))==player:
                in_a_row += 1
            elif  board.get((r,c)) != Opp and row_blank == 0:
                in_a_row += 1
                row_blank = 1
            else:
                if in_a_row < wwcd:
                    in_a_row = 0
                    row_blank = 0
        row_blank = 0
        if in_a_row >= wwcd:
            rows_tot += 1
    # print("Row", rows_tot)
    #Creating the starting points to go diagonally NW->SE
    NW_SE = []
    for c in range(1, scol+1):
        NW_SE.append((1,c))
    for r in range(2, srow+1):
        NW_SE.append((r,1))
    
    nwse_blank = 0
    nwse_tot = 0
    for start in NW_SE:
        row, col = start[0], start[1]
        in_a_nwse = 0
        while True:
            if row == srow+1 or col == scol+1:
                break
            # print(row,col)
            if board.get((row,col))==player:
                in_a_nwse += 1
            elif  board.get((row,col)) != Opp and nwse_blank == 0:
                in_a_nwse += 1
                nwse_blank = 1
            else:
                if in_a_nwse < wwcd:
                    in_a_nwse = 0
                    nwse_blank = 0
            row+=1
            col+=1
        nwse_blank = 0
        if in_a_nwse >= wwcd:
            nwse_tot += 1
    # print("NWSE", nwse_tot)
    #Creating the starting points to go diagonally SW-NE
    SW_NE = []
    for c in range(1, srow+1):
        SW_NE.append((c,1))
    for r in range(2, scol+1):
        SW_NE.append((srow,r))
    
    swne_blank = 0
    swne_tot = 0
    for start in SW_NE:
        row, col = start[0], start[1]
        in_a_swne = 0
        while True:
            if row == 0 or col == scol+1:
                break
            # print(row,col)
            if board.get((row,col))==player:
                in_a_swne += 1
            elif  board.get((row,col)) != Opp and swne_blank == 0:
                in_a_swne += 1
                swne_blank = 1
            else:
                if in_a_swne < wwcd:
                    in_a_swne = 0
                    swne_blank = 0
            row-=1
            col+=1
        swne_blank = 0
        if in_a_swne >= wwcd:
            swne_tot += 1
    # print("SWNE", swne_tot)      
    # print("col",cols_tot, " row",rows_tot, " NW->SE", nwse_tot," SW->NE", swne_tot)
    # We add on how many difreent ways we could possibly win
    return cols_tot + rows_tot + nwse_tot + swne_tot

### (2b) Compare A* to other algorithms
Next, play 10 games of Connect Four using your A* player and a random player and 10 games against the AB pruning player. In four or five paragraphs, report on the outcome. Did one player win more than the other? How often was the game a draw? How many moves did each player make? Were there situations where one player appeared to do better than the other? Given the outcome, are there other heuristics you would like to implement?

In [12]:
# Your code here.
won = 0
lost = 0
tie = 0

for i in range(10):
    ccc = ConnectFour(3,4,3)
    # This one is slower since it calcualtes right from the beginning I might have it play the edge or the middle, ive seen bots online play middle
    # outcome = ccc.play_game(alphabeta_player, random_player)
    outcome = ccc.play_game(Future_Astar, random_player)
    if outcome == 1: won+=1
    if outcome == -1: lost+=1
    if outcome == 0: tie+=1
    
print("Won%:", won/10, "Lost%:", lost/10, "tie%:", tie/10)

Won%: 1.0 Lost%: 0.0 tie%: 0.0


In [13]:
# alphabeta_player
# Your code here.
won = 0
lost = 0
tie = 0

for i in range(10):
    ccc = ConnectFour(3,4,3)
    # This one is slower since it calcualtes right from the beginning I might have it play the edge or the middle, ive seen bots online play middle
    # outcome = ccc.play_game(Future_Astar, alphabeta_player)
    outcome = ccc.play_game(alphabeta_player, Future_Astar)
    if outcome == 1: won+=1
    if outcome == -1: lost+=1
    if outcome == 0: tie+=1
    
print("Won%:", won/10, "Lost%:", lost/10, "tie%:", tie/10)

Won%: 1.0 Lost%: 0.0 tie%: 0.0
